In [1]:
import pandas as pd
from pandas import json_normalize
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_json('jsons/result1.json')

df = json_normalize(df['messages'])
channel_name = df['from'].dropna().iloc[0]
df = df[['date', 'text_entities']]